# Using LlamaIndex to Automate the fine-tuning of GPT-3.5-turbo on source documents

Primarly Extended from [this](https://colab.research.google.com/drive/1vWeJBXdFEObuihO7Z8ui2CAYkdHQORqo?usp=sharing) notebook, we'll take a look at how we can wrap this process into Chainlit and have our own dynamic fine-tuning machine!

In [141]:
# !pip install -q -U llama-index pypdf sentence-transformers ragas openai

In [ ]:
import os
from getpass import getpass

openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
!curl https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf --output hitchhikers.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3322k  100 3322k    0     0   663k      0  0:00:05  0:00:05 --:--:--  783k


The `ServiceContext` in LlamaIndex is a bundle of commonly used resources that are used during the indexing and querying stages of a LlamaIndex pipeline or application. 

It allows you to configure and customize various components of the pipeline. 

Here is a crash course on the ServiceContext:

### Concept:
The ServiceContext is a dataclass that contains the following components:

• LLM (Language Model): Used to generate natural language responses to queries.

• Prompt Helper: Helps with truncating and repacking text chunks to fit in the LLM's context window.

• Embedding Model: Generates vector representations of text.

• Node Parser: Converts documents into nodes.

• Callback Manager: Calls handlers on events and provides logging and tracing capabilities.

## Usage Pattern:

Configuring the service context:

You can directly construct a ServiceContext by passing in the desired components.

Alternatively, you can use the `ServiceContext.from_defaults` method to set default values for the components.

The `from_defaults`` method also allows you to configure specific modules, such as the LLM, Embedding Model, and Node Parser, by providing additional kwargs.

Setting global configuration:

 - You can set a service context as the global default for the entire LlamaIndex pipeline using the `set_global_service_context` function.
 - The global service context will be used as the default if not specified as a keyword argument in LlamaIndex functions.

Setting local configuration:

 - You can pass a service context to specific parts of the pipeline to override the default configuration.
 - For example, when creating a query engine, you can provide a service_context parameter to customize the behavior of the query engine.
 - By understanding and manipulating the ServiceContext, you can customize the behavior of the indexing process in LlamaIndex to suit your specific needs.

In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator

documents = SimpleDirectoryReader(
    input_files=["hitchhikers.pdf"]
).load_data()

# Shuffle the documents
import random

random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from a "
    "report on climate change and the oceans, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    documents[:50],
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

### Generative Questions with `gpt-3.5-turbo`

We can use the `generate_questions_from_nodes()` method of our dataset generator to produce a number of questions that will be used to fine-tune!

> NOTE: This cell will take ~30s-2min.

In [ ]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Let's take a peek and see what was created!

In [ ]:
questions[0]

'What did Zaphod find on the external monitor screens in the Horsehead Nebula?'

Now we can save our questions into a text file for later use.

In [ ]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluation Generator

Let's generate questions from a different segment of our documents in order to build a robust test for our RAQA.

In [ ]:
dataset_generator = DatasetGenerator.from_documents(
    documents[
        50:
    ],  # since we generated ~1 question for 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

Again, we'll use `gpt-3.5-turbo` to generate some questions!

In [ ]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Now we can save our results for evaluations later!

In [ ]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluating base `gpt-3.5-turbo`

We'll load up our evaluation questions and get to it!

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

This next cell is constructing our `VectorIndex` so we can move onto testing the base model.

This code sets up a vector store index using the `VectorStoreIndex` class, configures the service context with the OpenAI GPT-3.5 Turbo model and a context window size of 2048 tokens, and creates a query engine for performing similarity-based searches on the index. 

The query engine allows you to perform similarity-based searches on the index and retrieve the most relevant results for a given query.

In [ ]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3), context_window=2048
)

index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Here is where we're actually putting the model to the test!

Let's loop over the list of questions. 

For each question, uses the `query_engine` to query for a response. `query_engine.query` compares the query against the indexed documents, calculates similarity scores, ranks the documents, and returns the most relevant results based on the specified similarity_top_k value. 

The `query_engine.query` method takes a query string or document as input.

The query engine processes the query by converting it into a vector representation using the same embedding model that was used during the indexing process. This vector representation captures the semantic meaning of the query. The query engine then compares the vector representation of the query against the vector representations of the indexed documents using a similarity metric, such as cosine similarity. 

This metric calculates the similarity score between the query and each indexed document.

Based on the similarity scores, the query engine ranks the indexed documents in descending order, with the most similar documents receiving higher ranks. The query engine returns the top-k most similar results based on the similarity scores. The value of k is determined by the `similarity_top_k` parameter that was set when creating the query engine.

### `contexts.append([x.node.get_content() for x in response.source_nodes])`` is responsible for appending the content of each source node to the contexts list.

The response includes the source nodes, which are the relevant documents that were used to generate the answer, and the answer itself. The source nodes are stored in the contexts list, and the answers are stored in the answers list.


Let's break down the code:

• `response.source_nodes` refers to the list of source nodes returned by the query engine in response to a question.

• The code uses a list comprehension to iterate over each source node in `response.source_nodes`.

• For each source node, `x.node.get_content()`` is called to retrieve the content of the node.

• The content of each source node is then appended to a new list, which is then appended to the contexts list.

In [ ]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
print(response.source_nodes[0].node.get_content())

There  had  been  a  small  number  of  significant  letters  in  the  piles  of  
junk  ʹ  some  documents  from  the  council,  dated  three  years  earlier,  
relating  to  the  proposed  demolition  of  his  house,  and  some  other  
letters  about  the  setting  up  of  a  public  inquiry  into  the  whole  bypass  
scheme  in  the  area;  there  was  also  an  old  letter  from  Greenp eace,  the  
ecological  pressure  group  to  which  he  occasionally  made  
contributions,  asking  for  help  with  their  scheme  to  release  dolphins  
and  orcas  from  captivity,  and  some  postcards  from  friends,  vaguely  
complaining  that  he  never  got  in  touch  these  days.


In [ ]:
questions[32]

'What number did Deep Thought give as the Ultimate Answer in the context provided?'

In [ ]:
print(contexts[32][0])

"Yeah,"  he  said,  "only  there  was  a  man  in  a  mountain  wanted  to  
see  you.""I  met  him.""Yeah,  only  he  seemed  pretty  anxious  about  it,  you  know.""Yes,  I  met  him.""Yeah,  well  I  think  you  should  know  that.""I  do.I  met  him."The  man  paused  to  chew  a  little  gum.Then  he  clapped  Arthur  on  
the  back."OK,"  he  said,  "all  right.I'm  just  telling  you,  right?Good  night,  good  
luck,  win  awards.""What?"said  Arthur,  who  was  beginning  to  flounder  seriously  at  
this  point."Whatever.Do  what  you  do.Do  it  well."He  made  a  sort  of  clucking  
noise  with  whatever  he  was  chewing  and  then  some  vaguely  dynamic  
gesture."Why?"said  Arthur."Do  it  badly,"  said  the  man,  "who  cares?Who  gives  a  shit?"The  
blood  suddenly  seemed   to  pump  angrily  into  the  man's  face  and  he  
started  to  shout."Why  not  go  mad?"he  said."Go  away,  get  off  my  back  will  you,  
guy.Just  zark  off!!!""OK,  I'm  going,"  sa

In [ ]:
answers[32]

'The character received the award for "The Most Gratuitous Use Of The Word \'Fuck\' In A Serious Screenplay."'

Now that we've tested our model - let's evaluate it to see how it performed!

We're testing our model with the `ragas` framework - found [here](https://github.com/explodinggradients/ragas)

You'll notice that we're testing two primary metrics:

- [`answer_relevancy`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/answer_relevance.py#L32): This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.
- [`faithfulness`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/faithfulnes.py#L63): This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

Read more about their implementations [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

Again, these cells might take some time to complete - be patient!

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

/Users/harpreetsahota/miniconda3/envs/aimakerspace/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


evaluating with [answer_relevancy]


100%|██████████| 3/3 [01:19<00:00, 26.42s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [05:44<00:00, 114.96s/it]


{'ragas_score': 0.8777, 'answer_relevancy': 0.9246, 'faithfulness': 0.8352}


In [ ]:
base_eval = {'ragas_score': 0.8777, 'answer_relevancy': 0.9246, 'faithfulness': 0.8352}

### Leveraging `gpt-4` to improve our `gpt-3.5-turbo` base model!


In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [ ]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

This code sets up a vector store index using the `VectorStoreIndex` class, configures the service context with the OpenAI GPT-3.5 Turbo model and a context window size of 2048 tokens, and creates a query engine for performing similarity-based searches on the index.

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Again, this process will take a few minutes. 

While this is a powerful technique - it is unfortunately quite slow.

In [ ]:
for question in questions:
    response = query_engine.query(question)

In [ ]:
response

Response(response='The name of the ecological pressure group mentioned in the context is Greenpeace.', source_nodes=[NodeWithScore(node=TextNode(id_='81fe0d42-e805-45e6-a351-8d32241c7b1c', embedding=None, metadata={'page_label': '572', 'file_name': 'hitchhikers.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='91adb964-c6d6-402c-a798-fb29e0f9b9a4', node_type=None, metadata={'page_label': '572', 'file_name': 'hitchhikers.pdf'}, hash='c65b6a2f21376ac9c4e42fe472ee48394ff1998a5e77446c4bbc59669f1bbe15'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='50e8219c-e80a-433a-a924-19de9fe22e65', node_type=None, metadata={'page_label': '572', 'file_name': 'hitchhikers.pdf'}, hash='1c3c040adc656eeea3a66eedb3377bdcc59bb6c6302c7c7cb456f86eba7ad9d4')}, hash='bca507c26119c357d38f8e2d99577865a71ce9edd1f6d09849ce1d879d923c7a', text='There \xa0had \xa0been \xa0a \xa0small \xa0number \xa0of \xa0significa

### Creating the fine-tuning dataset

Now that we have a number of fine-tuning events from our `OpenAIFineTuningHandler()`, let's save them to a `.jsonl` file - the expected format for fine-tuning `gpt-3.5-turbo`!

In [ ]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 47 examples to finetuning_events.jsonl


In [ ]:
import openai
file_response = openai.File.create(file=open("finetuning_events.jsonl", "rb"), purpose='fine-tune')

In [ ]:
file_response

<File file id=file-e1DT1AWzFHjqntBwPXHOdirC at 0x1615c9670> JSON: {
  "object": "file",
  "id": "file-e1DT1AWzFHjqntBwPXHOdirC",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 200900,
  "created_at": 1693080563,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
import time

response = None

while not response:
  try:
    response = openai.FineTuningJob.create(training_file=file_response.id, model="gpt-3.5-turbo")
  except:
    time.sleep(5)

In [ ]:
response

<FineTuningJob fine_tuning.job id=ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK at 0x284430590> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693080603,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-uVzc7jXuBij843gam1Xdonxx",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-e1DT1AWzFHjqntBwPXHOdirC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [ ]:
training_id = response.id

In [ ]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK at 0x28678daf0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693080603,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-uVzc7jXuBij843gam1Xdonxx",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-e1DT1AWzFHjqntBwPXHOdirC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [ ]:
from IPython.display import clear_output

while openai.FineTuningJob.retrieve(training_id).status == "running":
  clear_output(wait=True)
  time.sleep(5)
  print(openai.FineTuningJob.list_events(id=training_id, limit=10))

print("Done!")

{
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-q8c2gpgXa0NHfLpZ4n6BOLzF",
      "created_at": 1693081421,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-V3C4WMqxuYCTgN7QNNmI2WAo",
      "created_at": 1693081419,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::7ru6l1bi",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-tXV7SEGD65xeJ8Ypb683PURc",
      "created_at": 1693081412,
      "level": "info",
      "message": "Step 140/141: training loss=0.16",
      "data": {
        "step": 140,
        "train_loss": 0.1555028259754181,
        "train_mean_token_accuracy": 0.9485294222831726
      },
      "type": "metrics"
    },
    {
      "object": "fine_tuning.j

In [ ]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK at 0x2842931d0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-Uz1vMnKp6RVIhYcPzxsXC6GK",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693080603,
  "finished_at": 1693081421,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7ru6l1bi",
  "organization_id": "org-uVzc7jXuBij843gam1Xdonxx",
  "result_files": [
    "file-vDEXotv2n2fQhJhK4PXnTF3M"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-e1DT1AWzFHjqntBwPXHOdirC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 160344
}

In [ ]:
ft_model_id = openai.FineTuningJob.retrieve(training_id).fine_tuned_model

### Evaluating the fine-tuned model

Now that we've fine-tuned our model on the `gpt-4` enhanced question answers - let's see how it performs on our `raga` evaluation!

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [ ]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:53<00:00, 17.88s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [03:05<00:00, 61.70s/it]


{'ragas_score': 0.8092, 'answer_relevancy': 0.9400, 'faithfulness': 0.7104}


In [ ]:
ft_eval = {'ragas_score': 0.8092, 'answer_relevancy': 0.9400, 'faithfulness': 0.7104}

In [ ]:
base_eval

{'ragas_score': 0.8777, 'answer_relevancy': 0.9246, 'faithfulness': 0.8352}

### Exploring Differences

Now we can compare the outputs of the two models!

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
print(questions[12])

What did the bird claim reverse engineering enables them to do?


In [ ]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** The bird claimed that reverse engineering enables them to quickly analyze and understand the technology of a spaceship. This allows them to anticipate when a spaceship will arrive and confidently secure a lift without depending on the spaceship's discretion.

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** The bird claimed that reverse engineering allows them to bypass the need to wait for a spaceship to pass through their galactic sector and evaluate whether or not to offer someone a ride. Instead, they can ascertain that a spaceship will provide transportation and facilitate it.

In [ ]:
metric_list = ["answer_relevancy", "faithfulness"]

for metric in metric_list:
  print("Base model", metric, ":", base_eval[metric])
  print("Fine-tuned model", metric, ":", ft_eval[metric])
  print(f"Improvement {metric} : {(ft_eval[metric] - base_eval[metric])*100:.2f}%")
  print()

Base model answer_relevancy : 0.9246
Fine-tuned model answer_relevancy : 0.94
Improvement answer_relevancy : 1.54%

Base model faithfulness : 0.8352
Fine-tuned model faithfulness : 0.7104
Improvement faithfulness : -12.48%

